# Import libraries

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from helpers import parse_variables, get_risk_level, hi_gauss_blob_risk_fun, blob_risk_fun, NW_risk_fun, square_risk_fun, map_to_color
from matplotlib.colors import LinearSegmentedColormap
import importlib.util
from k_means_constrained import KMeansConstrained


import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import tensorflow as tf

from helpers import parse_variables, get_risk_level, map_to_color, simulate_quant_trait
from models import ols_regression, manhattan_linear, gc
from deep_learning_models import abyss, deep_abyss

from scipy.stats import t
from scipy import stats
from scipy.stats import entropy

from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

from tensorflow.keras import Input, Model, layers, regularizers
from tensorflow.keras.layers import Input, Dense
import random

from collections import Counter
from math import floor


2024-08-28 18:09:37.195548: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-28 18:09:37.198955: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-28 18:09:37.208222: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-28 18:09:37.236049: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-28 18:09:37.236090: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-28 18:09:37.251295: I tensorflow/core/platform/cpu_feature_guard.cc:

In [2]:
dict = parse_variables('geno_simulation.txt')
G = int(dict['G'])
L = int(dict['L'])
c = int(dict['c'])
k = int(dict['k'])
M = float(dict['M'])
HWE = int(dict['HWE'])
nr_humans = int(dict['nr_humans'])
nr_snps = int(dict['nr_snps'])
bottleneck_nr = int(dict['bottleneck_nr'])
# Thresholds
very_rare_threshold_L = float(dict['very_rare_threshold_L'])
very_rare_threshold_H = float(dict['very_rare_threshold_H'])

rare_threshold_L = float(dict['rare_threshold_L'])
rare_threshold_H = float(dict['rare_threshold_H'])

common_threshold_L = float(dict['common_threshold_L'])
common_threshold_H = float(dict['common_threshold_H'])

number_of_snps = (G*L)/2 # one loci per chromosome
number_of_individuals = c*k*k

In [3]:
very_rare = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/01_veryrare_genotype_AF_{very_rare_threshold_L}_{very_rare_threshold_H}.pkl")
rare = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/01_rare_genotype_AF_{rare_threshold_L}_{rare_threshold_H}.pkl")
common = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/01_common_genotype_AF_{common_threshold_L}_{common_threshold_H}.pkl")

In [4]:
very_rare = very_rare.rename(columns=lambda x: 'VR' + x)/2
rare = rare.rename(columns=lambda x: 'R' + x)/2
common = common.rename(columns=lambda x: 'C' + x)/2
complete = pd.concat([common, rare, very_rare], axis=1)
complete = ((complete*2)-1)

In [5]:
# Read bottleneck

In [6]:
path_bottle = f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/phenotype/abyss_bottleneck"
bottle_file = [f for f in os.listdir(path_bottle) if int(f.split("_")[2]) ==  bottleneck_nr][0]

In [7]:
elapsed_time_bottleneck = float(bottle_file.split('_')[3].split('seconds')[0])

In [8]:
bottle = pd.read_pickle(f"{path_bottle}/{bottle_file}")

In [9]:
complete['pop'] = bottle['cluster']

In [10]:
nr_snps

2000

# Divide in LD blocks

In [11]:
for pop in bottle['cluster'].unique():
    temp_complete = complete[complete['pop']==pop]
    temp_complete = temp_complete.drop('pop', axis = 1)
    genos = temp_complete.T
    sample_size = nr_snps
    n_components = 15
    num_clus = round(genos.shape[0] / sample_size)
    size_clus = int(genos.shape[0]/num_clus)
    size_min = size_clus - round(size_clus / 5)
    size_max = size_clus + round(size_clus / 5)
    # Calculate the number of clusters based on sample size
    # Standardize the data
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(genos)

    # Perform PCA with n components
    pca = PCA(n_components=n_components)
    principal_components = pca.fit_transform(df_scaled)
    
    # Create a new DataFrame to store the principal components
    pc_columns = [f'PC{i+1}' for i in range(n_components)]
    df_pca = pd.DataFrame(data=principal_components, columns=pc_columns)
    
    # Apply constrained K-Means clustering
    clf = KMeansConstrained(
        n_clusters=num_clus,
        size_min=size_min,
        size_max=size_max,
        random_state=0
    )
    clf.fit_predict(np.array(df_pca))

    # Assign cluster labels to genotypic data
    count = Counter(clf.labels_)
    empty = []
    sample_size_temp = floor(len(clf.labels_)/sample_size)
    for val in count.values():
        multi = floor(val/sample_size_temp)+1
        list_to_sample = multi*list(range(sample_size_temp))
        random.shuffle(list_to_sample)
        empty = empty + list_to_sample[0:val]
        
    genos['clusters_k_means'] = empty # shuffled
    #genos['clusters_k_means'] = clf.labels_ # clustered

    PATH_output = f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/LD_blocks/{pop}"
    os.system(f"rm -rf {PATH_output}")
    os.makedirs(PATH_output, exist_ok=True)

    for num in genos.clusters_k_means.unique():
        print(num)
        to_save = genos.loc[genos['clusters_k_means'] == num]
        to_save = to_save.drop(columns=['clusters_k_means']).T
    
        # Calculate minimum and maximum MAF for the cluster
        tot_mafs = []
        for snp_id in to_save:
            try:
                try:
                    num_maj = to_save[[snp_id]].value_counts()[1]
                except Exception as e:
                    num_maj = 0
                try:
                    num_het = to_save[[snp_id]].value_counts()[0]
                except Exception as e:
                    num_het = 0
                try:
                    num_min = to_save[[snp_id]].value_counts()[-1]
                except Exception as e:
                    num_min = 0
                total_humans = num_maj + num_het + num_min
                maf = (num_min*2 + num_het)/(total_humans*2)
    
            except Exception as e:
                print(e)
                print(f"snp {snp_id} has a problem")
    
            tot_mafs.append(maf)
    
        max_maf = np.round(max(tot_mafs), 5)
        min_maf = np.round(min(tot_mafs), 5)
        size = to_save.shape[1]
    
        # Save the processed cluster data
        to_save.to_pickle(f"{PATH_output}/{num}_{size}_maf_{min_maf}_{max_maf}.pkl")

    path_one_hot_genotype = f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/LD_blocks_one_hot/{pop}"
    os.system(f"rm -rf {path_one_hot_genotype}")
    os.makedirs(path_one_hot_genotype, exist_ok = True)

    path_lds = f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/LD_blocks/{pop}"
    ld_files = os.listdir(path_lds)
    for ld_file in ld_files:
        path_ld_file = path_lds + "/" + ld_file
        ld_complete = pd.read_pickle(path_ld_file)
    
        # Create db_minor
        db_minor = ld_complete.copy()
        db_minor = db_minor.applymap(lambda x: 1 if x == -1.0 else 0)
        
        # Create db_het
        db_het = ld_complete.copy()
        db_het = db_het.applymap(lambda x: 1 if x == 0.0 else 0)
        
        # Create db_major
        db_major = ld_complete.copy()
        db_major = db_major.applymap(lambda x: 1 if x == 1.0 else 0)
        
    
        db_minor.to_pickle(f"{path_one_hot_genotype}/{ld_file.split('.pkl')[0]}_db_minor.pkl")
        db_het.to_pickle(f"{path_one_hot_genotype}/{ld_file.split('.pkl')[0]}_db_het.pkl")
        db_major.to_pickle(f"{path_one_hot_genotype}/{ld_file.split('.pkl')[0]}_db_major.pkl")

1
3
2
0


0.4

In [19]:
db_major

snps,CV811_AF_0.2125,CV813_AF_0.22875,CV827_AF_0.233125,CV832_AF_0.224375,CV838_AF_0.23125,CV846_AF_0.2275,CV850_AF_0.2125,CV854_AF_0.22625,CV889_AF_0.21125,CV897_AF_0.211875,...,VRV9805_AF_0.020625,VRV9823_AF_0.0275,VRV9845_AF_0.036875,VRV9885_AF_0.036875,VRV9896_AF_0.046875,VRV9940_AF_0.0175,VRV9942_AF_0.035,VRV9950_AF_0.024375,VRV9955_AF_0.026875,VRV9966_AF_0.035625
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,0,1
1,1,1,1,1,0,0,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,0,1,1,0,...,1,1,1,1,1,1,1,1,1,1
3,0,1,1,1,1,1,0,1,1,1,...,1,1,1,1,1,1,1,1,0,1
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,0,1,1,1
796,1,1,1,1,1,0,1,0,1,1,...,1,1,1,1,1,1,1,1,1,1
797,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1
798,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,1,1,1,1,1
